In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style="color:powderblue;"> The Red Wine Quality Prediction </h1>
<h3 style="color:pink;"> Step by step explanation </h3>

In [ ]:
#Importing all the necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from collections import Counter
from IPython.core.display import display, HTML
sns.set_style('darkgrid')

<h2 style="color:powderblue;"> EDA </h2>

In [ ]:
#loading dataset
rwine = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

#Shape
print(rwine.shape)
print("----------------------------")
print(rwine.head(5))
print("----------------------------")
print(rwine.info())
print("----------------------------")
print(rwine.describe())

<h3 style="color:pink;"> Data is clean no null values present </h3>

In [ ]:
rwine["quality"].unique

<h1 style="color:powderblue;"> Plotting Charts and Observing them </h1>

In [ ]:
#Here we see that fixed acidity does not give any specification to classify the quality.
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'fixed acidity', data = rwine)


In [ ]:

fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'volatile acidity', data = rwine)
plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'citric acid', data = rwine)

In [ ]:
fig = plt.figure(figsize = (20,20))
sns.lineplot(x = 'alcohol', y = 'citric acid', data = rwine)

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'residual sugar', data = rwine)

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'chlorides', data = rwine)

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(x = 'quality', y = 'total sulfur dioxide', data = rwine)

In [ ]:
sns.set_style("whitegrid");
sns.pairplot(rwine, hue="quality",height=3);
plt.show()

<h1 style="color:pink;"> Encoding our target variable </h1>

Why Encoding is Important?
We want to classify our data. And it only make sense if our data is catagorical.
So, we are splitting the data in Good or Bad.

In [ ]:
rwine['quality'].value_counts()

<h3 style

In [ ]:
bins = (2, 6, 8)
labels = ['bad', 'good']
rwine['quality'] = pd.cut(x = rwine['quality'], bins = bins, labels = labels)

In [ ]:
rwine["quality"].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
rwine['quality'] = labelencoder_y.fit_transform(rwine['quality'])
rwine['quality'] 

<h1 style="color:powderblue;"> Dividing the columns into input and target variable.</h1>

In [ ]:
X = rwine.drop('quality', axis = 1).values
y = rwine['quality'].values.reshape(-1,1)

<h3>The Data is divided using basic 80% of traning and 20% test set </h3>

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 42)

<h3 style="color:pink;"> Feature Scaling:  It is the process of standerizing the data. We do this to our data so that while building a machine learning model, our model is not biased towards a particular feature of the dataset. </h3>

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [ ]:
X_train_scaled

<h2 style="color:powderblue;"> Logistic Regression (Sigmoid/S shaped) </h2>

In [ ]:
# Fitting Logistic Regression to the Training set
#Taken the maximum iteration possible, assigned default penality,alowed constant adding
from sklearn.linear_model import LogisticRegression
classifier_lr = LogisticRegression(C=1, fit_intercept=True, max_iter=1000, penalty = 'l2', solver='liblinear')
#The numpy.ravel() functions returns contiguous flattened array.
classifier_lr.fit(X_train_scaled, y_train.ravel())

<h2 style="color:pink;"> Cross Validation: It is a resampling procedure used to evaluate machine learning models on a limited data sample. </h1>

In [ ]:
# Predicting Cross Validation Score
#taking 10 fold cross-validation
cv_lr = cross_val_score(estimator = classifier_lr, X = X_train_scaled, y = y_train.ravel(), cv = 10)
print("CV: ", cv_lr.mean())

y_pred_lr_train = classifier_lr.predict(X_train_scaled)
accuracy_lr_train = accuracy_score(y_train, y_pred_lr_train)
print("Training set: ", accuracy_lr_train)

y_pred_lr_test = classifier_lr.predict(X_test_scaled)
accuracy_lr_test = accuracy_score(y_test, y_pred_lr_test)
print("Test set: ", accuracy_lr_test)

<h3 style="color:lightgreen;">Logistic Regressor gave us 88% Accuracy which is good but we will try to find better if possible.</h3>

<h3 style="color:powderblue;"> Decison Tree: It is a supervised learning algorithm used for classification. <h3>

**Gini Coefficient: It shows the information gain and help to split the tree in more efficient way.**

In [ ]:
# Fitting classifier to the Training set
#gini coefficient
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'gini', max_features=6, max_leaf_nodes=400, random_state = 33)
dt.fit(X_train_scaled, y_train.ravel())

In [ ]:
# Predicting Cross Validation Score
cv_dt = cross_val_score(estimator = dt, X = X_train_scaled, y = y_train.ravel(), cv = 10)
print("CV: ", cv_dt.mean())

y_pred_train = dt.predict(X_train_scaled)
accuracy_dt_train = accuracy_score(y_train, y_pred_train)
print("Training set: ", accuracy_dt_train)

y_pred_dt_test = dt.predict(X_test_scaled)
accuracy_dt_test = accuracy_score(y_test, y_pred_dt_test)
print("Test set: ", accuracy_dt_test)

<h3 style="color:pink;"> Random Forest :   It consists of a large number of individual decision trees that operate as an ensemble. Each tree in the random forest spits out a class prediction and the class with the most votes becomes our model’s prediction. It also remove the problem created by over fitting of data</h3>

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion = 'entropy', max_features = 4, n_estimators = 800, random_state=33)
rf.fit(X_train_scaled, y_train.ravel())

In [ ]:
 #Use the forest's predict method on the test data
predictions = rf.predict(X_test_scaled)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Predicting Cross Validation Score
cv_rf = cross_val_score(estimator = rf, X = X_train_scaled, y = y_train.ravel(), cv = 10)
print("CV: ", cv_rf.mean())

y_pred_rf_train = rf.predict(X_train_scaled)
accuracy_rf_train = accuracy_score(y_train, y_pred_rf_train)
print("Training set: ", accuracy_rf_train)

y_pred_rf_test = rf.predict(X_test_scaled)
accuracy_rf_test = accuracy_score(y_test, y_pred_rf_test)
print("Test set: ", accuracy_rf_test)

<h3 style="color:pink;">Support Vector Machine: Support Vector Machine” (SVM) is a supervised machine learning algorithm that can be used for both classification or regression challenges </h3>

In [ ]:
from sklearn.svm import SVC

<h3 style="color:powder Blue;">Linear </h3>

In [ ]:
svc = SVC(kernel = 'linear')
svc.fit(X_train_scaled, y_train.ravel())
pred_svc = svc.predict(X_test_scaled)
print(classification_report(y_test, pred_svc))

In [ ]:
# Predicting Cross Validation Score
cv_svm_linear = cross_val_score(estimator = svc, X = X_train_scaled, y = y_train.ravel(), cv = 10)
print("CV: ", cv_svm_linear.mean())

y_pred_svm_linear_train = svc.predict(X_train_scaled)
accuracy_svm_linear_train = accuracy_score(y_train, y_pred_svm_linear_train)
print("Training set: ", accuracy_svm_linear_train)

y_pred_svm_linear_test = svc.predict(X_test_scaled)
accuracy_svm_linear_test = accuracy_score(y_test, y_pred_svm_linear_test)
print("Test set: ", accuracy_svm_linear_test)

<h3 style="color:pink;"> Radial Basis Function </h3>

In [ ]:
# Fitting classifier to the Training set
svm_kernel = SVC(kernel = 'rbf', C = 10, tol = 0.001, gamma = 'scale')
svm_kernel.fit(X_train_scaled, y_train.ravel())


In [ ]:
# Predicting Cross Validation Score
cv_svm_kernel = cross_val_score(estimator = svm_kernel, X = X_train_scaled, y = y_train.ravel(), cv = 10)
print("CV: ", cv_svm_kernel.mean())

y_pred_svm_kernel_train = svm_kernel.predict(X_train_scaled)
accuracy_svm_kernel_train = accuracy_score(y_train, y_pred_svm_kernel_train)
print("Training set: ", accuracy_svm_kernel_train)

y_pred_svm_kernel_test = svm_kernel.predict(X_test_scaled)
accuracy_svm_kernel_test = accuracy_score(y_test, y_pred_svm_kernel_test)
print("Test set: ", accuracy_svm_kernel_test)

In [ ]:
svc = SVC(kernel = 'rbf')
svc.fit(X_train_scaled, y_train.ravel())
pred_svc = svc.predict(X_test_scaled)
print(classification_report(y_test, y_pred_svm_kernel_test))

<h4 style="color:lightgreen;">We can see that we have everything better in RBC than we had in linear. Model accuracy, precision everything improved</h4>

**Also, The model which gives the best result is Rndom Forest Classifier**

At the very end, I would just like to mention I have take help of several notebooks to understand and write my own.